In [1]:
%load_ext autoreload
%autoreload 2
import pynwb
import os
import numpy as np

# DataJoint and DataJoint schema
import datajoint as dj

dj.config["filepath_checksum_size_limit"] = 1 * 1024**2

## We also import a bunch of tables so that we can call them easily
from spyglass.common import (
    ElectrodeGroup,
    Electrode,
    FirFilterParameters,
    IntervalList,
    Session,
    Task,
    TaskEpoch,
    Nwbfile,
    AnalysisNwbfile,
    NwbfileKachery,
    AnalysisNwbfileKachery,
    interval_list_contains,
    interval_list_contains_ind,
    interval_list_excludes,
    interval_list_excludes_ind,
    interval_list_intersect,
    get_electrode_indices,
)

from spyglass.lfp.v1 import (
    LFPElectrodeGroup,
    LFPSelection,
    LFPV1,
)
from spyglass.lfp.lfp_merge import LFPOutput


from spyglass.lfp.analysis.v1.lfp_band import LFPBandSelection,LFPBandV1
# from spyglass.lfp_band.lfp_band_merge import LFPBandOutput
import os
import warnings
from tqdm import tqdm
warnings.simplefilter("ignore", category=DeprecationWarning)
warnings.simplefilter("ignore", category=ResourceWarning)
warnings.simplefilter("ignore", category=UserWarning)

[2023-11-08 13:59:08,488][INFO]: Connecting sambray@lmf-db.cin.ucsf.edu:3306
[2023-11-08 13:59:08,521][INFO]: Connected sambray@lmf-db.cin.ucsf.edu:3306


## Define Animal Sessions

In [3]:
nwb_file_names = [
    #Olive
    "Olive20220624_.nwb",
    "Olive20220627_.nwb",
    "Olive20220628_.nwb",
    "Olive20220629_.nwb",
    "Olive20220630_.nwb",
    "Olive20220701_.nwb",
    "Olive20220702_.nwb",
    "Olive20220705_.nwb",
    "Olive20220706_.nwb",
    #Yoshi
    "Yoshi20220509_.nwb",
    "Yoshi20220510_.nwb",
    "Yoshi20220511_.nwb",
    "Yoshi20220519_.nwb",
    "Yoshi20220525_.nwb",
    #Winnie
    "Winnie20220712_.nwb",
    "Winnie20220713_.nwb",
    "Winnie20220714_.nwb",
    "Winnie20220715_.nwb",
    "Winnie20220716_.nwb",
    #Wallie
    "Wallie20220911_.nwb",
    "Wallie20220912_.nwb",
    "Wallie20220913_.nwb",
    "Wallie20220916_.nwb",
    #Totoro
    "Totoro20220524_.nwb",
    "Totoro20220525_.nwb",
    "Totoro20220526_.nwb",
    "Totoro20220527_.nwb",
    "Totoro20220530_.nwb",
    "Totoro20220531_.nwb",
    "Totoro20220602_.nwb",
    "Totoro20220603_.nwb",
    "Totoro20220606_.nwb",
    "Totoro20220621_.nwb",
    #Odins
    "Odins20220107_.nwb",
    "Odins20220110_.nwb",
    "Odins20220111_.nwb",
    "Odins20220113_.nwb",
    "Odins20220118_.nwb",
    "Odins20220119_.nwb",
    "Odins20220123_.nwb",
    "Odins20220124_.nwb",
    # "Odins20220126_.nwb", ???
    #Banner
    "Banner20211221_.nwb",
    "Banner20211222_.nwb",
    "Banner20211227_.nwb",
    "Banner20211228_.nwb",
    "Banner20211230_.nwb",
    "Banner20211231_.nwb",
    "Banner20220103_.nwb",
    "Banner20220105_.nwb",
    "Banner20220123_.nwb",
    "Banner20220124_.nwb",
    "Banner20220126_.nwb",
]

nwb_file_names.extend([
    #Olive
    "Olive20220707_.nwb",
    "Olive20220708_.nwb",
    "Olive20220711_.nwb",
    #Yoshi
    "Yoshi20220513_.nwb",
    "Yoshi20220516_.nwb",
    "Yoshi20220517_.nwb",
    "Yoshi20220518_.nwb",
    #Winnie
    "Winnie20220717_.nwb",
    "Winnie20220718_.nwb",
    "Winnie20220719_.nwb",
    "Winnie20220720_.nwb",
    "Winnie20220722_.nwb",
    #Wallie
    "Wallie20220914_.nwb",
    "Wallie20220915_.nwb",
    "Wallie20220922_.nwb",
    #Totoro
    "Totoro20220607_.nwb",
    "Totoro20220610_.nwb",
    "Totoro20220613_.nwb",
    "Totoro20220614_.nwb",
    "Totoro20220615_.nwb",
    "Totoro20220616_.nwb",
    "Totoro20220617_.nwb",
    #Odins
    "Odins20220226_.nwb",
    #Banner
    "Banner20220215_.nwb",
    "Banner20220216_.nwb",
    "Banner20220223_.nwb",
    "Banner20220224_.nwb",
    "Banner20220225_.nwb",
])
    
# np.isnan(np.mean([]))
animal_names = ["Wallie","Winnie","Olive","Totoro","Yoshi","Dan"]
animal_names = ["Bilbo","Winnie","Frodo"]
from spyglass.common import Session
nwb_file_names = []
[nwb_file_names.extend((Session() & {"subject_id":animal}).fetch("nwb_file_name")) for animal in animal_names]

nwb_file_names = ['Winnie20220714_.nwb']

#write permission?
analysis_path = "/stelmo/nwb/analysis/"
for nwb_file in nwb_file_names:
    if not os.path.isdir(analysis_path+nwb_file.split('_')[0]):
        print(f'No analysis yet for: {nwb_file}')
    elif not os.access(analysis_path+nwb_file.split('_')[0],os.W_OK):
        print(f'No write permissions for: {nwb_file}nwb_file')


In [9]:
animal_names = ["Wallie","Winnie","Olive","Totoro","Yoshi","Dan","Bilbo","Winnie","Frodo"]
nwb_file_names = []
interval_list_names = []

for animal in animal_names:
    nwb_file_names.extend((LFPV1 * Session() & {"subject_id":animal}).fetch("nwb_file_name"))
    interval_list_names.extend((LFPV1 * Session() & {"subject_id":animal}).fetch("target_interval_list_name"))

## Populate

In [10]:
# nwb_file_names = ["Totoro20220602_.nwb",]
# interval_list_name = "pos 3 valid times"
filter_name = "Ripple 150-250 Hz"
# desired sampling rate
lfp_band_sampling_rate = 1000
missing_lfp = []



############################################
for nwb_file_name, interval_list_name in tqdm(zip(nwb_file_names, interval_list_names)):

    
    basic_key = {"nwb_file_name": nwb_file_name,'target_interval_list_name':interval_list_name}
    #check that lfp exists for this animal
    if len(LFPV1() & basic_key)==0:
        missing_lfp.append(basic_key)
        continue
    if len(LFPV1 & basic_key)>1:
        electrode_group_names = (LFPV1() & basic_key).fetch("lfp_electrode_group_name")
        electrode_group_name =[x for x in electrode_group_names if nwb_file_name.split('202')[0] in x][0]
        basic_key["lfp_electrode_group_name"] = electrode_group_name
        
    #get lfp's sampling rate
    lfp_sampling_rate = (LFPV1() & basic_key ).fetch1(
        "lfp_sampling_rate"
    )
    
    #get lfp filter sampling_rate
    lfp_filter_sampling_rate = (LFPV1() & basic_key).fetch1(
        "filter_sampling_rate"
    )
    
    #get lfp filter name
    lfp_filter_name = (LFPV1() & basic_key).fetch1(
        "filter_name"
    )
    
    # make lfp selection key
    lfp_s_key = basic_key.copy()
    lfp_s_key["target_interval_list_name"] = interval_list_name
    lfp_s_key["filter_name"] = lfp_filter_name
    lfp_s_key["filter_sampling_rate"] = lfp_filter_sampling_rate
    
    # get the electrodes the lfp was run on
    lfp_electrode_group_name = (LFPV1 & lfp_s_key).fetch1('lfp_electrode_group_name')
    lfp_eg_key = {
            "nwb_file_name": nwb_file_name,
            "lfp_electrode_group_name": lfp_electrode_group_name,
            }
    electrode_list = list(np.asarray((LFPV1 & basic_key).fetch_nwb()[0]['lfp'].electrodes.to_dataframe().index))
    
    # remake lfp selection key
    lfp_s_key = basic_key.copy()
    lfp_s_key["target_interval_list_name"] = interval_list_name
    lfp_s_key["filter_name"] = lfp_filter_name
    lfp_s_key["filter_sampling_rate"] = lfp_filter_sampling_rate
    
    # set the reference to -1 to indicate no reference for all channels
    ref_elect = [-1]
    
    # we also need the uuid for the LFP object
    lfp_merge_id = (LFPOutput.LFPV1 & lfp_s_key).fetch1('merge_id')
    
    
    # #make Band selection table entry
    LFPBandSelection().set_lfp_band_electrodes(
            nwb_file_name=nwb_file_name,
            lfp_merge_id=lfp_merge_id,
            electrode_list=electrode_list,
            filter_name=filter_name,
            interval_list_name=interval_list_name,
            reference_electrode_list=ref_elect,
            lfp_band_sampling_rate=lfp_band_sampling_rate,
        )
    
    lfp_band_key = (
        LFPBandSelection
        & {
            "lfp_merge_id": lfp_merge_id,
            "filter_name": filter_name,
            "lfp_band_sampling_rate": lfp_band_sampling_rate,
            "target_interval_list_name":interval_list_name,
        }
    ).fetch1("KEY")
    if LFPBandV1() & lfp_band_key:
        continue #already populated
    # lfp_band_key['lfp_electrode_group_name']="full_probe"
    LFPBandV1().populate(LFPBandSelection()&lfp_band_key)

0it [00:00, ?it/s][2023-11-08 14:03:15,780][WARNING]: Skipped checksum for file with hash: 32d257cc-18c3-dc6a-f383-91f7d20f67eb, and path: /stelmo/nwb/analysis/Wallie20220911/Wallie20220911_1MLN1R56OB.nwb
1it [00:01,  1.14s/it][2023-11-08 14:03:16,899][WARNING]: Skipped checksum for file with hash: ff6de7e3-7fbc-5cbd-397d-dca30069b95b, and path: /stelmo/nwb/analysis/Wallie20220911/Wallie20220911_W3Z8U3HB7F.nwb
2it [00:01,  1.10it/s][2023-11-08 14:03:17,638][WARNING]: Skipped checksum for file with hash: 4b15d951-2f3b-e456-aee7-d875aa162d6a, and path: /stelmo/nwb/analysis/Wallie20220911/Wallie20220911_LDJT1IUE6A.nwb
3it [00:02,  1.24it/s][2023-11-08 14:03:18,325][WARNING]: Skipped checksum for file with hash: bd08e2a0-2393-9ebc-69e9-8a8657b0939b, and path: /stelmo/nwb/analysis/Wallie20220911/Wallie20220911_7HH21BMRB9.nwb
4it [00:03,  1.35it/s][2023-11-08 14:03:18,973][WARNING]: Skipped checksum for file with hash: e22a802f-e7c9-3ff1-e822-9ce840325924, and path: /stelmo/nwb/analysis/Wall

Writing new NWB file Bilbo20230725_JSOWF430SR.nwb


888it [10:49,  1.37it/s]


ValueError: 'input_index_bounds' [1087515, 1087515] is not strictly increasing

In [10]:
lfp_band_key

(LFPV1 & basic_key).fetch_nwb()[0]['lfp']

[2023-10-20 08:18:10,755][WARNING]: Skipped checksum for file with hash: 1f41ed1f-df73-533d-6c95-5cb459607516, and path: /stelmo/nwb/analysis/Bilbo20230725/Bilbo20230725_JRH9I6411O.nwb


filtered data pynwb.ecephys.ElectricalSeries at 0x140401433184672
Fields:
  comments: no comments
  conversion: 1.0
  data: <HDF5 dataset "data": shape (2016507, 14), type "<i2">
  description: filtered data
  electrodes: electrodes <class 'hdmf.common.table.DynamicTableRegion'>
  interval: 1
  offset: 0.0
  resolution: -1.0
  timestamps: <HDF5 dataset "timestamps": shape (2016507,), type "<f8">
  timestamps_unit: seconds
  unit: volts

## Check missing LFP's

In [13]:
print(missing_lfp)

'Banner20220225_.nwb'